## Enhanced featrue engineer model

This model will add engineered features for the original question, in addition to the lemmatized question.

Engineered two different types of features,

1. n_gram similarity between each pair of questions
2. min/max/avg distance between words in a single question. Currently using the following metrics,
  * euclidean
  * cosine
  * city block or manhattan
  
**Pipeline**
1. Stack questions
2. Clean questions - now lower cases all words to better lemmatize proper nouns
3. UNION
    1. n_gram similarity
    2. min/max/avg distance
4. Lemmatize questions
5. UNION
    1. n_gram similarity
    2. min/max/avg distances
6. UNION together both sets of features
7. Avearge Ensemble
    1. Random Forrest
    2. XGBClassifier

In [1]:
# data manipulation
import utils
import pandas as pd
import numpy as np

# modeling
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from xgboost import XGBClassifier

In [2]:
X_train = utils.load('X_train')
y_train = utils.load('y_train')
MEM_PATH = '../data/transform_memory'

## Pre-Process pipes

In [3]:
clean_pipe = Pipeline(
    [
        ('stack', FunctionTransformer(utils.stack_questions, validate=False)),
        ('clean', FunctionTransformer(utils.clean_questions, validate=False))
    ],
    memory = MEM_PATH
)

lemma_pipe = Pipeline(
    [
        ('lemma', FunctionTransformer(utils.apply_lemma, validate=False))
    ],
    memory = MEM_PATH
)

## Cos similarity of TF-IDF vector plus NMF topics

In [4]:
nmf_pipe = Pipeline(
    [
        ('nmf', NMF(n_components=5)),
        ('unstack', FunctionTransformer(utils.unstack_questions, validate=True))
    ]
)

cos_pipe = Pipeline(
    [
        ('cos', FunctionTransformer(utils.calc_cos_sim_stack, validate=False))
    ]
)

nmf_cos_pipe = Pipeline(
    [
        ('clean', clean_pipe),
        ('lemma', lemma_pipe),
        ('tf', TfidfVectorizer()),
        ('feats', FeatureUnion(
            [
                ('nmf_pipe', nmf_pipe),
                ('cos_pipe', cos_pipe)
            ]
        )),
        ('xgb', XGBClassifier(n_estimators=500, n_jobs=-1, random_state=42))
    ]
)
# X_transform = pipe.fit_transform(X_train)

## Feature Engineering Pipes

In [5]:
# feature engineering pipes
single_question_pipe = Pipeline(
    [
        ('dist', FunctionTransformer(utils.add_min_max_avg_distance_features, validate=False)),
        ('unstack', FunctionTransformer(utils.unstack_questions, validate=False))
    ]
)

pair_question_pipe = Pipeline(
    [
        ('ngram_sim', FunctionTransformer(utils.calc_ngram_similarity, kw_args={'n_grams':[1, 2, 3]}, validate=False))
    ]
)

# clean text pipe
clean_text_pipe = Pipeline(
    [
        ('clean', clean_pipe),
        ('feats', FeatureUnion(
            [
                ('pair', pair_question_pipe),
                ('single', single_question_pipe)
            ]
        ))
    ]
)

# lemma pipe
lemma_text_pipe = Pipeline(
    [
        ('clean', clean_pipe),
        ('lemma', lemma_pipe),
        ('feats', FeatureUnion(
            [
                ('pair', pair_question_pipe),
                ('single', single_question_pipe)
            ]
        ))
    ]
)

## XGBoost

In [6]:

XGB_pipe = Pipeline(
    [
        ('feats', FeatureUnion(
            [
                ('clean_features', clean_text_pipe),
                ('lemma_pipe', lemma_text_pipe)
            ]
        )),
        ('xgb', XGBClassifier(n_estimators=500, n_jobs=-1, random_state=42))
    ]
)

## Random Forrest

In [7]:
RF_pipe = Pipeline(
    [
        ('feats', FeatureUnion(
            [
                ('clean_features', clean_text_pipe),
                ('lemma_pipe', lemma_text_pipe)
            ]
        )),
        ('rf', RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42))
    ]
)

## Ensemble

In [8]:
# weighting based on individual AUC
## cos_sim = 0.799173
## xgboost = 0.846923
## rf = 0.868202

total = 0.799173 + 0.846923 + 0.868202
cos_sim_weight = 0.799173 / total
xgb_weight = 0.846923 / total
rf_weight = 0.868202 / total

weights = [cos_sim_weight, xgb_weight, rf_weight]

# nmf_cos = utils.load('cos_sim_tfidf_model')
# xgb = utils.load('xgb_feat_eng_model')
# rf = utils.load('rf_feat_eng_model')

estimators = [('cos_sim', nmf_cos_pipe), ('xgb', XGB_pipe), ('rf', RF_pipe)]
vc = VotingClassifier(estimators, voting='soft', n_jobs=1, weights=weights) 
## don't think this works with what I want to do

## Cross-validate

In [9]:
# X_transform = pre_process_pipe.transform(X_train)

skf = StratifiedKFold(n_splits=3, random_state=42)
cv = cross_validate(vc, 
               X_train[:10000], 
               y_train[:10000], 
               cv=skf, 
               n_jobs=1, 
               scoring=('accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'neg_log_loss'),
               verbose=100)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  ................................................................
[CV]  , accuracy=0.7381523695260948, precision=0.6622807017543859, recall=0.6074014481094127, f1=0.6336550566512797, roc_auc=0.8162834782481563, neg_log_loss=-0.49677170945574356, total= 4.5min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.9min remaining:    0.0s
[CV]  ................................................................
[CV]  , accuracy=0.72997299729973, precision=0.6474137931034483, recall=0.604669887278583, f1=0.6253122398001666, roc_auc=0.8163473393756387, neg_log_loss=-0.4961386748642456, total= 4.4min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 21.8min remaining:    0.0s
[CV]  ................................................................
[CV]  , accuracy=0.7479747974797479, precision=0.6741767764298093, recall=0.6264090177133655, f1=0.649415692821369, roc_auc=0.8273306887658249, neg_log_loss=-0.48

## Log results

In [10]:
results_df = utils.load('results')

results_df = results_df.drop(index='ensemble_rf_xgb_cos_sim', errors='ignore')
results_df = results_df.append(utils.log_scores(cv, 'ensemble_rf_xgb_cos_sim'))
results_df

,avg_accuracy,std_accuracy,avg_precision,std_precision,avg_recall,std_recall,avg_f1,std_f1,avg_auc,std_auc,avg_log_loss,std_log_loss
"mvp (tf-idf, nmf(5), xgboost)",0.700345,0.000466,0.661571,0.000461,0.385736,0.002493,0.487325,0.001983,0.740593,0.001647,0.568958,0.001288
mvp (+ lemma),0.696787,0.001055,0.649977,0.003057,0.387424,0.003230,0.485464,0.002485,0.738037,0.001362,0.572483,0.000815
cos_sim_model,0.710200,0.000830,0.658748,0.002578,0.446336,0.002215,0.532120,0.001306,0.746769,0.001279,0.565250,0.000963
feat_eng_model,0.743614,0.002021,0.664102,0.003502,0.618400,0.001553,0.640434,0.002281,0.821070,0.001428,0.489465,0.001141
feat_eng_model_lemma_fix,0.744356,0.002107,0.664513,0.004333,0.621357,0.000901,0.642201,0.001609,0.822197,0.001710,0.488131,0.001342
ensemble_rf_xgb,0.779000,0.002740,0.697794,0.004357,0.708157,0.001912,0.702935,0.003148,0.863334,0.001438,0.441784,0.001107
cos_sim_tfidf_model,0.729511,0.001216,0.661680,0.002219,0.547188,0.001744,0.599010,0.001703,0.800271,0.001291,0.512085,0.001299
feat_eng_model_lemma_clean,0.763927,0.002404,0.676166,0.003904,0.692113,0.001128,0.684044,0.002549,0.846923,0.001643,0.456929,0.001410
rf_feat_eng_model_lemma_clean,0.783667,0.002260,0.708853,0.003681,0.702725,0.001666,0.705774,0.002658,0.868202,0.001148,0.436197,0.000640
ensemble_rf_xgb_cos_sim,0.738700,0.007359,0.661290,0.010948,0.612827,0.009669,0.636128,0.009994,0.819987,0.005193,0.493703,0.003901


In [11]:
utils.save(results_df, 'results')

## Results

Wow! The feature engineering shows a significant jump in AUC from 0.8 to 0.82.

In [9]:
vc = VotingClassifier([('rf', rf), ('xgb', xgb)], voting='soft')
vc.fit(X_transform, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_we...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [10]:
y_probs = vc.predict_proba(X_transform)[:, 1]
class_errors_df = utils.ground_truth_analysis(y_train, y_probs)
class_errors_df.head()

,gt,prob,diff
0,0,0.271811,-0.271811
1,0,0.331749,-0.331749
2,0,0.158016,-0.158016
3,0,0.000237,-0.000237
4,0,0.034689,-0.034689


In [ ]:
lemma_pipe = Pipeline(
    [
        ('stack', FunctionTransformer(utils.stack_questions, validate=False)),
        ('clean', FunctionTransformer(utils.clean_questions, validate=False)),
        ('lemma', FunctionTransformer(utils.apply_lemma, validate=False)),
    ]
)
X_train_lemma = lemma_pipe.transform(X_train)

## Top false negative errors

In [ ]:
fn_idx = class_errors_df.sort_values('diff', ascending = False).head().index
for idx in fn_idx:
    print('Prob:', y_probs[idx])
    print()
    print(X_train.iloc[idx].question1)
    print(X_train.iloc[idx].question2)
    print()
    print('Lemma--------')
    print()
    print(X_train_lemma[idx*2])
    print(X_train_lemma[idx*2+1])
    print()
    print('Feature Space------')
    print(X_transform[idx])
    print('-------------------------------------------')
    print()

## Top false positive errors

In [ ]:
fn_idx = class_errors_df.sort_values('diff').head().index
for idx in fn_idx:
    print('Prob:', y_probs[idx])
    print()
    print(X_train.iloc[idx].question1)
    print(X_train.iloc[idx].question2)
    print()
    print('Lemma--------')
    print()
    print(X_train_lemma[idx*2])
    print(X_train_lemma[idx*2+1])
    print()
    print('Feature Space------')
    print(X_transform[idx])
    print('-------------------------------------------')
    print()

## Next Steps

1. The false negative errors appear to be related to numbers. 
  * Either stop removing numebers in the clean text proces, or create another set of features.
2. The false positives seem very tricky. Found the Facebook InferSent model which embeds sentences to 4096 dimension.
  * Could see if the distances between some of the false positive examples is different in this space.
3. Could look at alignments of the two questions. (I think nltk)
4. Add different word embeddings to get a different nuance, or even use the vector only data set from Spacy.

In [18]:
nmf_cos_proba = nmf_cos.predict_proba(X_nmf_cos_feat)[:, 1]
xgb_proba = xgb.predict_proba(X_feat_eng_transform)[:, 1]
rf_proba = rf.predict_proba(X_feat_eng_transform)[:, 1]

In [13]:
nmf_cos_proba = nmf_cos_proba * cos_sim_weight
xgb_proba = xgb_proba * xgb_weight
rf_proba = rf_proba * rf_weight
ensemble_proba = nmf_cos_proba + xgb_proba + rf_proba
ensemble_proba[:10]

array([0.39567994, 0.34479889, 0.17470645, 0.00103656, 0.07662025,
       0.45168266, 0.29587881, 0.14138014, 0.0013542 , 0.68504003])

In [15]:
from sklearn import metrics

In [20]:
print(f'Ensemble: {metrics.roc_auc_score(y_train, ensemble_proba)}')
print(f'Nmf_cos: {metrics.roc_auc_score(y_train, nmf_cos_proba)}')
print(f'XGB: {metrics.roc_auc_score(y_train, xgb_proba)}')
print(f'RF: {metrics.roc_auc_score(y_train, rf_proba)}')

Ensemble: 0.9846300420170292
Nmf_cos: 0.8056886449194024
XGB: 0.8540229521257536
RF: 0.9999764283543164


In [21]:
print(f'Ensemble: {metrics.log_loss(y_train, ensemble_proba)}')
print(f'Nmf_cos: {metrics.log_loss(y_train, nmf_cos_proba)}')
print(f'XGB: {metrics.log_loss(y_train, xgb_proba)}')
print(f'RF: {metrics.log_loss(y_train, rf_proba)}')

Ensemble: 0.32102219836966456
Nmf_cos: 0.5069449937264294
XGB: 0.4495672382949828
RF: 0.12308486086868513
